# Pretrained LLMにもとづく学習と予測


In [1]:
!pip install transformers==4.24.0
!pip install datasets==2.10.1 fugashi==1.2.1 ipadic==1.0.0
!pip install unidic-lite
#!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.9/599.9 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 54.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 955.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
import numpy as np
import datasets
from transformers import BertForSequenceClassification, BertJapaneseTokenizer,MobileBertForSequenceClassification
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from transformers import TrainingArguments
from transformers import Trainer
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

#sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-v3", num_labels=3)
sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-v2", num_labels=3)
#sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese", num_labels=3)
sc_model = sc_model.to(device)

# https://dev.classmethod.jp/articles/huggingface-jp-text-classification/
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")

#tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-large-japanese")
#sc_model = MobileBertForSequenceClassification.from_pretrained("ysakuramoto/mobilebert-ja") # 文書分類の場合
#sc_model = sc_model.to(device)

tsukurepo_df = pd.read_csv('tsukurepo_df.csv', encoding='ms932', sep=',',skiprows=0)
tsukurepo_df.sample(frac=1)
tsukurepo_texts = tsukurepo_df['tsukurepo'].values.tolist()
labels = tsukurepo_df['keyword'].values
uniq_l = np.unique(labels)
label_dic = {w:i for i,w in enumerate(uniq_l)}
label_dic_inv = {i:w for i,w in enumerate(uniq_l)}
#label_ids = np.array([label_dic[w] for w in labels])


text_data=[]
for l,txt in zip(labels,tsukurepo_texts):
    #print(txt)
    txt = txt.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""}))
    text_data.append([txt,label_dic[l]])


#label_df = pd.DataFrame(label_ids.reshape(-1,1),columns=['label'])
tsukurepo = pd.DataFrame(text_data,columns=['text','label'])
#tsukurepo = pd.concat([text_df,label_df],axis=1)
#tsukurepo=tsukurepo.rename(columns = {'tsukurepo':'text'})
train_idx = int(len(tsukurepo)*0.75)
train_df = tsukurepo.iloc[:train_idx,:]
test_df = tsukurepo.iloc[train_idx:,:]

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)


train_data = datasets.Dataset.from_pandas(train_df[['text', 'label']])
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])



eval_data = datasets.Dataset.from_pandas(test_df[['text', 'label']])
eval_data = eval_data.map(tokenize, batched=True, batch_size=len(eval_data))
eval_data.set_format("torch", columns=["input_ids", "label"])




def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

# data, data/logs などはcolaboratoryで自動的に作成される

training_args = TrainingArguments(output_dir = "./data",logging_dir = "./data/logs",num_train_epochs =10,per_device_train_batch_size = 5,
                                  per_device_eval_batch_size = 32,
                                  warmup_steps=500,weight_decay=0.001,evaluation_strategy = "epoch")

trainer = Trainer(
    model = sc_model,  # 使用するモデルを指定
    args = training_args,  # TrainingArgumentsの設定
    compute_metrics = compute_metrics,  # 評価用の関数
    train_dataset = train_data,  # 訓練用のデータ
    eval_dataset = eval_data  # 評価用のデータ
)

trainer.train()
print(trainer.evaluate())

model_path = "./data/"


sc_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

cuda:0


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/config.json
Model config BertConfig {
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32768
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggi

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese/snapshots/32fc2539e890ac278af0e56efb9aa6570f777345/vocab.txt
loading file spiece.model from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese/snapshots/32fc2539e890ac278af0e56efb9aa6570f777345/tokenizer_config.json


...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 32
The following co

Configuration saved in ./data/config.json


{'eval_loss': 2.0790910720825195, 'eval_accuracy': 0.6268656716417911, 'eval_runtime': 1.6012, 'eval_samples_per_second': 83.688, 'eval_steps_per_second': 3.123, 'epoch': 10.0}


Model weights saved in ./data/pytorch_model.bin
tokenizer config file saved in ./data/tokenizer_config.json
Special tokens file saved in ./data/special_tokens_map.json


('./data/tokenizer_config.json',
 './data/special_tokens_map.json',
 './data/vocab.txt',
 './data/added_tokens.json')

In [ ]:
preds_output = trainer.predict(eval_data)
print(preds_output)
preds = preds_output[0].argmax(-1)
counter=0
for p,t in zip(preds,test_df['label']):

  if p==t:
    counter+=1
print(counter/len(test_df))

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 134
  Batch size = 32


PredictionOutput(predictions=array([[-2.712747  ,  6.053155  , -2.9266999 ],
       [-2.8223028 ,  6.081216  , -2.904806  ],
       [-2.7493474 ,  6.0281725 , -2.9832585 ],
       [-2.8446732 ,  6.057896  , -2.9275563 ],
       [-2.7797062 ,  6.0734124 , -3.0024955 ],
       [-2.6912296 ,  6.0762343 , -2.9699502 ],
       [-2.7738497 ,  6.077728  , -2.9570727 ],
       [-2.6418004 , -2.2129314 ,  5.024942  ],
       [-2.8080914 ,  6.0541215 , -3.0117562 ],
       [-4.3397565 ,  3.496576  ,  0.67442805],
       [-2.0521593 , -1.9447825 ,  3.215645  ],
       [-2.7409995 ,  6.05683   , -2.8834794 ],
       [-2.7323804 ,  6.047606  , -2.907355  ],
       [-1.3244678 ,  3.3771203 , -2.576658  ],
       [-2.854283  ,  6.055466  , -2.93061   ],
       [-3.2575088 , -3.092987  ,  6.35967   ],
       [-3.8167245 ,  0.3782725 ,  3.3917763 ],
       [-2.8583782 ,  6.0789733 , -2.931583  ],
       [-2.8821976 ,  6.071741  , -2.881531  ],
       [-0.7735584 ,  0.12308484,  0.49847576],
       [-3.